In [1]:
from obspy.core.event import read_events
import obspy
import numpy as np
import pandas as pd
from scipy.special import sph_harm
import matplotlib.pyplot as plt
import os.path
import shutil
from obspy import UTCDateTime
from netCDF4 import Dataset
from matplotlib import cm
from skimage.filters import gaussian
from mpl_toolkits.basemap import Basemap
from itertools import chain

from scipy.stats import qmc
import h5py

dpi = 300
FigFormat = "png"

def draw_map(map):

    # draw a shaded-relief image
    map.shadedrelief(scale=0.2)
    
    # lats and longs are returned as a dictionary
    lats = map.drawparallels(np.linspace(-90, 90, 13))
    lons = map.drawmeridians(np.linspace(-180, 180, 13))

    # keys contain the plt.Line2D instances
    lat_lines = chain(*(tup[1][0] for tup in lats.items()))
    lon_lines = chain(*(tup[1][0] for tup in lons.items()))
    all_lines = chain(lat_lines, lon_lines)
    
    # cycle through these lines and set the desired style
    for line in all_lines:
        line.set(linestyle='-', alpha=0.3, color='w')

In [2]:
# Dim = (81*23)*3+9-81*2
# N = 10000
# scalerange = 1
# sampler = qmc.LatinHypercube(d=Dim)
# sample = sampler.random(n=N)
# l_bounds = [-scalerange]*Dim
# u_bounds = [scalerange]*Dim
# sample_scaled = qmc.scale(sample, l_bounds, u_bounds)
# np.save('sampling_array_%d_prem_iso' %N,sample_scaled)

# plt.figure(figsize=(5, 5))
# plt.scatter(sample_scaled[:, 0], sample_scaled[:, 1])
# plt.xlabel("d0")
# plt.ylabel("d1")
# plt.title(f"Latin Hypercube Sampling (n = {1000})")

In [2]:
sample_scaled = np.load('sampling_array_10000_prem_iso.npy')

In [3]:
num_models = 10000
SourceDepthRange = [0,800]
source_dims = (num_models, 9)
with h5py.File('../Data/PREM_Sources.h5', "w") as data_hdf5:
    source_data = data_hdf5.create_dataset("source", source_dims, dtype='float32')

    for imodel in np.arange(0,10000):
        print(f"Processing model {imodel}")
        para_index = 0
        # update event source (9 parameters)
        EventDepth = (sample_scaled[imodel,para_index]+1)/2*(SourceDepthRange[1]-SourceDepthRange[0])+SourceDepthRange[0]
        source_data[imodel,0] = EventDepth
        para_index += 1
        MomentTensor = sample_scaled[imodel,para_index:para_index+6]*1e24
        source_data[imodel,1:7] = np.array(MomentTensor)
        para_index += 6
        EventLat = (sample_scaled[imodel,para_index]+1)/2*180+(-90)
        source_data[imodel,7] = EventLat
        para_index += 1
        EventLon = (sample_scaled[imodel,para_index]+1)/2*360+(-180)
        source_data[imodel,8] = EventLon
        para_index += 1

Processing model 0
Processing model 1
Processing model 2
Processing model 3
Processing model 4
Processing model 5
Processing model 6
Processing model 7
Processing model 8
Processing model 9
Processing model 10
Processing model 11
Processing model 12
Processing model 13
Processing model 14
Processing model 15
Processing model 16
Processing model 17
Processing model 18
Processing model 19
Processing model 20
Processing model 21
Processing model 22
Processing model 23
Processing model 24
Processing model 25
Processing model 26
Processing model 27
Processing model 28
Processing model 29
Processing model 30
Processing model 31
Processing model 32
Processing model 33
Processing model 34
Processing model 35
Processing model 36
Processing model 37
Processing model 38
Processing model 39
Processing model 40
Processing model 41
Processing model 42
Processing model 43
Processing model 44
Processing model 45
Processing model 46
Processing model 47
Processing model 48
Processing model 49
Processing

In [ ]:
ExampleInputDir = '../Runs/ExampleSphericalHarmonicsPREM/input'

SourceDepthRange = [0,800]
DvpRange = 0.02
DvsRange = 0.02
DrhoRange = 0.01

RadiusList = [6371.0, 6356.0, 6356.0, 6346.6, 6346.6, 6291.0, 6291.0, 6151.0, 6151.0, 5971.0, 5971.0, 5771.0, 5771.0, 5701.0, 5701.0, 5600.0, 5600.0, 3630.0, 3630.0, 3480.0, 3480.0, 1221.5, 1221.5, 0.0]
RadiusNotationList = ['6371.0','6356.0a', '6356.0b', '6346.6a', '6346.6b', '6291.0a', '6291.0b', '6151.0a', '6151.0b', '5971.0a', '5971.0b', '5771.0a', '5771.0b', '5701.0a', '5701.0b', '5600.0a', '5600.0b', '3630.0a', '3630.0b', '3480.0a', '3480.0b', '1221.5a', '1221.5b', '0.0']

for imodel in np.arange(580,10000):

    para_index = 0

    ModeName = 'LatinSphericalHarmonicsPREM%04d' %imodel
    EventParDir='../Runs/%s' %(ModeName)  
    if not os.path.exists(EventParDir):
        os.makedirs(EventParDir)
    print(EventParDir, " created")

    if not os.path.exists(EventParDir+'/input'):
        os.makedirs(EventParDir+'/input')

    # copy parameter file
    # shutil.copy(ExampleInputDir+'/TomoNet_LowerMantle__10s.e',EventParDir+'/input/')

    shutil.copy(ExampleInputDir+'/inparam.model.yaml',EventParDir+'/input/')

    shutil.copy(ExampleInputDir+'/inparam.nr.yaml',EventParDir+'/input/')

    shutil.copy(ExampleInputDir+'/inparam.advanced.yaml',EventParDir+'/input/')

    shutil.copy(ExampleInputDir+'/inparam.source.yaml',EventParDir+'/input/')
    # update event source (9 parameters)
    EventDepth = (sample_scaled[imodel,para_index]+1)/2*(SourceDepthRange[1]-SourceDepthRange[0])+SourceDepthRange[0]
    para_index += 1
    MomentTensor = sample_scaled[imodel,para_index:para_index+6]*1e24
    para_index += 6
    EventLat = (sample_scaled[imodel,para_index]+1)/2*180+(-90)
    para_index += 1
    EventLon = (sample_scaled[imodel,para_index]+1)/2*360+(-180)
    para_index += 1

    with open(EventParDir+'/input/inparam.source.yaml','r') as file:
        filetxt = file.read()
    filetxt = filetxt.replace("latitude_longitude: [90, 0]", "latitude_longitude: [%.2f, %.2f]" %(EventLat, EventLon))
    filetxt = filetxt.replace("depth: 200.0e0", "depth: %.1fe3" %(EventDepth))
    filetxt = filetxt.replace("data: [1e10, 1e10, 1e10, 1e10, 1e10, 1e10]", "data: [%e, %e, %e, %e, %e, %e]"  %(MomentTensor[0], MomentTensor[1], MomentTensor[2], MomentTensor[3], MomentTensor[4], MomentTensor[5]))
    with open(EventParDir+'/input/inparam.source.yaml','w') as file:
        file.write(filetxt)

    shutil.copy(ExampleInputDir+'/inparam.output.yaml',EventParDir+'/input/')

    shutil.copy(ExampleInputDir+'/Synthetic_Stations_Ball.txt',EventParDir+'/input/')

    # # generate random model
    ### Real spherical harmonics
    coeff = {}

    ModelCoeff = dict()
    ModelCoeff['variable'] = []
    ModelCoeff['Radius'] = []
    ModelCoeff['l'] = []
    ModelCoeff['m'] = []
    ModelCoeff['Value'] = []

    l_max = 8
    RadiusList = [6371.0, 6356.0, 6356.0, 6346.6, 6346.6, 6291.0, 6291.0, 6151.0, 6151.0, 5971.0, 5971.0, 5771.0, 5771.0, 5701.0, 5701.0, 5600.0, 5600.0, 3630.0, 3630.0, 3480.0, 3480.0, 1221.5, 1221.5, 0.0]
    RadiusNotationList = ['6371.0','6356.0a', '6356.0b', '6346.6a', '6346.6b', '6291.0a', '6291.0b', '6151.0a', '6151.0b', '5971.0a', '5971.0b', '5771.0a', '5771.0b', '5701.0a', '5701.0b', '5600.0a', '5600.0b', '3630.0a', '3630.0b', '3480.0a', '3480.0b', '1221.5a', '1221.5b', '0.0'] 

    for Radius in RadiusNotationList:
        coeff[Radius] = {}

    for variable in ['dvp','dvs','drho']:
        for Radius in RadiusNotationList[0:-1]:

            if Radius=='3480.0b' and variable=='dvs':
                continue
            if Radius=='1221.5a' and variable=='dvs':
                continue

            
            for l in range(0,l_max+1):
                for m in np.arange(-l,l+1):
                    name = '%s_%s_%s' %(variable, l, m)
                    # print(l,m)
                    ModelCoeff['variable'].append(variable)
                    ModelCoeff['Radius'].append(Radius)
                    ModelCoeff['l'].append(l)
                    ModelCoeff['m'].append(m)

                    if variable == 'dvp':
                        Val = sample_scaled[imodel,para_index]*DvpRange  # Latin Hypercube Sampling
                    elif variable == 'dvs':
                        Val = sample_scaled[imodel,para_index]*DvsRange  # Latin Hypercube Sampling
                    elif variable == 'drho':
                        Val = sample_scaled[imodel,para_index]*DrhoRange  # Latin Hypercube Sampling

                    ModelCoeff['Value'].append(Val)
                    coeff[Radius][name] = Val 
                    para_index += 1


    df = pd.DataFrame(data=ModelCoeff)
    df.to_pickle(EventParDir+"/Spherical_Harmonics.pkl")

    # Make sure RADIUS and Coordinates are ascendingly sorted
    grid_lat = np.linspace(-90, 90, 181)
    grid_lon = np.linspace(-180, 180, 361)
    grid_lat.sort()
    grid_lon.sort()

    LON, LAT = np.meshgrid(grid_lon, grid_lat)
    DvpMLTomo = np.zeros([len(grid_lat), len(grid_lon), 2])
    DvsMLTomo = np.zeros([len(grid_lat), len(grid_lon), 2])
    DrhoMLTomo = np.zeros([len(grid_lat), len(grid_lon), 2])

    for TopDiscontinuity, BotDiscontinuity in zip(RadiusNotationList[0::2], RadiusNotationList[1::2]):
        print('TopDiscontinuity: ', TopDiscontinuity, 'BotDiscontinuity: ', BotDiscontinuity)
        grid_radius = np.array([float(BotDiscontinuity.strip('a')), float(TopDiscontinuity.strip('b'))])

        for variable in ['dvp','dvs','drho']:
            if TopDiscontinuity=='3480.0b' and variable=='dvs': #Skip the outer core vs setting
                continue

            for i, SlicingRadius in enumerate([BotDiscontinuity, TopDiscontinuity]):
                
                # define the center value 
                if SlicingRadius == '0.0':
                    if variable == 'dvp':
                        DvpMLTomo[:,:,i] = 0.0
                    elif variable == 'dvs':
                        DvsMLTomo[:,:,i] = 0.0
                    elif variable == 'drho':
                        DrhoMLTomo[:,:,i] = 0.0
                    continue

                # initiate TomoSum
                TomoSum = np.zeros([len(grid_lat),len(grid_lon)])

                for l in range(0,l_max+1):
                    for m in np.arange(-l,l+1):
                        # print('l, m = ', l, m)
                        name = '%s_%s_%s' %(variable, l, m)
                        Y_grid = sph_harm(m, l, np.radians(LON-180), np.radians(90-LAT))

                        if m < 0:
                            Y_grid = np.sqrt(2) * (-1)**(-m) * Y_grid.imag
                        elif m > 0:
                            Y_grid = np.sqrt(2) * (-1)**m * Y_grid.real

                        TomoSum[:,:] = TomoSum[:,:] + coeff[SlicingRadius][name] * Y_grid
                if variable == 'dvp':
                    DvpMLTomo[:,:,i] = TomoSum[:,:]
                elif variable == 'dvs':
                    DvsMLTomo[:,:,i] = TomoSum[:,:]
                elif variable == 'drho':
                    DrhoMLTomo[:,:,i] = TomoSum[:,:]
            
    

    # # Fig Preparation
    # dpi = 200
    # fig = plt.figure(figsize=(3.5,3),dpi=200)
    # ax = fig.add_subplot(111)

    # map = Basemap(projection='moll',lon_0=0,resolution='l') # moll Projection
    # PLOT = map.pcolormesh(LON, LAT, TomoSum, latlon=True, cmap=plt.get_cmap('jet'))
    # cbar = plt.colorbar(PLOT, ax=ax, shrink=0.5)
    # ax.set_title('Depth Slice at %s m to degrees %d' %(SlicingDepth, l_max))
    # draw_map(map)
    # map.drawcoastlines(linewidth=0.1)
        # print('DvpMLTomo min: ', DvpMLTomo.min(), 'DvpMLTomo max: ', DvpMLTomo.max())

        # make discountinity nc file for 
        NCName = "degree8_random%sto%s.nc" %(TopDiscontinuity, BotDiscontinuity)
        # write to file
        if os.path.exists(EventParDir+'/input/'+NCName):
            os.remove(EventParDir+'/input/'+NCName)

        nc = Dataset(EventParDir+'/input/'+NCName, 'w')
        nc.createDimension('nlat', size=len(grid_lat))
        nc.createDimension('nlon', size=len(grid_lon))
        nc.createDimension('nradius', size=len(grid_radius))
        nc.createVariable('latitude', float, dimensions=('nlat'))
        nc['latitude'][:] = grid_lat
        nc.createVariable('longitude', float, dimensions=('nlon'))
        nc['longitude'][:] = grid_lon
        nc.createVariable('radius', float, dimensions=('nradius'))
        nc['radius'][:] = grid_radius
        nc.createVariable('dvp', float, dimensions=('nlat', 'nlon','nradius'))
        nc['dvp'][:,:,:] = DvpMLTomo[:,:,:]
        nc.createVariable('dvs', float, dimensions=('nlat', 'nlon','nradius'))
        nc['dvs'][:,:,:] = DvsMLTomo[:,:,:]
        nc.createVariable('drho', float, dimensions=('nlat', 'nlon','nradius'))
        nc['drho'][:,:,:] = DrhoMLTomo[:,:,:]

        if imodel == 0:
            nc['dvp'][:,:,:] = np.zeros(np.shape(DvpMLTomo[:,:,:]))
            nc['dvs'][:,:,:] = np.zeros(np.shape(DvsMLTomo[:,:,:]))
            nc['drho'][:,:,:] = np.zeros(np.shape(DrhoMLTomo[:,:,:]))
            nc.close()
            continue

        nc.close()



In [50]:
for TopDiscontinuity, BotDiscontinuity in zip(RadiusNotationList[0::2], RadiusNotationList[1::2]):
    print(TopDiscontinuity, BotDiscontinuity)

6371.0 6356.0a
6356.0b 6346.6a
6346.6b 6291.0a
6291.0b 6151.0a
6151.0b 5971.0a
5971.0b 5771.0a
5771.0b 5701.0a
5701.0b 5600.0a
5600.0b 3630.0a
3630.0b 3480.0a
3480.0b 1221.5a
1221.5b 0.0


In [19]:
DrhoMLTomo[:,:,0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
DrhoMLTomo[:,:,1]

array([[-0.00429728, -0.00429728, -0.00429728, ..., -0.00429728,
        -0.00429728, -0.00429728],
       [-0.00482948, -0.00481697, -0.00480432, ..., -0.00485402,
        -0.00484183, -0.00482948],
       [-0.00534158, -0.00531678, -0.0052917 , ..., -0.00539035,
        -0.00536611, -0.00534158],
       ...,
       [-0.00932567, -0.00927601, -0.00922692, ..., -0.00942666,
        -0.00937589, -0.00932567],
       [-0.0107507 , -0.01072539, -0.01070046, ..., -0.01080236,
        -0.01077635, -0.0107507 ],
       [-0.01216559, -0.01216559, -0.01216559, ..., -0.01216559,
        -0.01216559, -0.01216559]])

In [10]:
df.loc[(df['Radius'] == '1221.5b')]

,variable,Radius,l,m,Value
1782,dvp,1221.5b,0,0,-0.004928
1783,dvp,1221.5b,1,-1,-0.004049
1784,dvp,1221.5b,1,0,0.004222
1785,dvp,1221.5b,1,1,0.002000
1786,dvp,1221.5b,2,-2,0.001813
...,...,...,...,...,...
5422,drho,1221.5b,8,4,0.000770
5423,drho,1221.5b,8,5,0.001933
5424,drho,1221.5b,8,6,-0.001784
5425,drho,1221.5b,8,7,0.000985


In [12]:
df.loc[(df['Radius'] == '0.0')]

,variable,Radius,l,m,Value
